In [1]:
import os
import os.path as osp

import numpy as np
import pandas as pd

from mmengine.config import Config
from mmdet.apis import DetInferencer
from pycocotools.coco import COCO

In [2]:
# custom 설정
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass",
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

root = "../../dataset/"
test_ann = "test.json"

saved_epoch = 1
batch_size = 4
score_thr = 0.05
device = "cuda:0"

work_dir = "./work_dirs/faster_rcnn_r50_fpn_1x_trash"
checkpoint_path = osp.join(work_dir, f"epoch_{saved_epoch}.pth")

# config file 들고오기
cfg = Config.fromfile("configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py")
cfg.default_scope = "mmdet"

# dataset config 수정
ds = cfg.test_dataloader["dataset"] if "dataset" in cfg.test_dataloader else cfg.test_dataloader
ds.metainfo = dict(classes=classes)
ds.data_root = root
ds.ann_file = test_ann
ds.data_prefix = dict(img="")
cfg.test_dataloader.dataset.pipeline[1]["scale"] = (512, 512)

cfg.randomness = dict(seed=2025, deterministic=False)
cfg.model.roi_head.bbox_head.num_classes = len(classes)

In [3]:
# test image info
coco = COCO(osp.join(root, test_ann))
img_ids = coco.getImgIds()
imgs = coco.loadImgs(img_ids)

def _img_path(im):
    return osp.join(root, "", im["file_name"]) if "" \
        else osp.join(root, im["file_name"])

img_paths = [_img_path(im) for im in imgs]
file_names = [im["file_name"] for im in imgs]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
# DetInferencer를 이용한 batch inference
inferencer = DetInferencer(
    model=cfg,
    weights=checkpoint_path,
    device=device
)

results = inferencer(
    img_paths,
    batch_size=batch_size,
    return_datasamples=True,
    no_save_vis=True,
)

Loads checkpoint by local backend from path: ./work_dirs/faster_rcnn_r50_fpn_1x_trash/epoch_1.pth
12/10 10:08:39 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


Output()

/data/ephemeral/home/venv/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [5]:
# submission 양식에 맞게 output 후처리
prediction_strings = []

for data_sample in results["predictions"]:
    inst = data_sample.pred_instances
    bboxes = inst.bboxes.cpu().numpy() if hasattr(inst.bboxes, "cpu") else np.asarray(inst.bboxes)
    scores = inst.scores.cpu().numpy() if hasattr(inst.scores, "cpu") else np.asarray(inst.scores)
    labels = inst.labels.cpu().numpy() if hasattr(inst.labels, "cpu") else np.asarray(inst.labels)

    keep = scores >= score_thr
    bboxes, scores, labels = bboxes[keep], scores[keep], labels[keep]

    ps = []
    for (x1, y1, x2, y2), sc, lb in zip(bboxes, scores, labels):
        ps.extend([str(int(lb)), f"{float(sc):.6f}",
                   f"{float(x1):.1f}", f"{float(y1):.1f}",
                   f"{float(x2):.1f}", f"{float(y2):.1f}"])
    prediction_strings.append(" ".join(ps))

submission = pd.DataFrame({
    "PredictionString": prediction_strings,
    "image_id": file_names,
})
save_path = osp.join(work_dir, f"output.csv")
submission.to_csv(save_path, index=False)
print(f"Saved: {save_path}")
print(submission.head())

Saved: ./work_dirs/faster_rcnn_r50_fpn_1x_trash/output.csv
                                    PredictionString       image_id
0  7 0.987106 615.8 509.0 937.5 1024.0 7 0.974858...  test/0000.jpg
1  7 0.750555 99.8 11.6 501.7 262.9 7 0.655290 32...  test/0001.jpg
2  1 0.427157 92.0 262.9 331.1 601.1 1 0.398445 2...  test/0002.jpg
3  1 0.171403 25.4 425.5 161.8 686.9 1 0.168202 3...  test/0003.jpg
4  7 0.430008 158.2 387.7 915.3 732.2 7 0.384609 ...  test/0004.jpg
